In [1]:
# !pip install transformers
# !pip install datasets
from transformers import AutoModelForSeq2SeqLM, T5ForConditionalGeneration,AutoTokenizer
from datasets import load_dataset, load_metric

C:\Users\Jayathilaga\miniconda3\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [36]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [37]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from datasets import load_dataset

In [5]:
path_file='generated_job_descriptions_and_linked_resumes.csv'
dataset_generated=load_dataset('csv', data_files=[path_file],split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/Jayathilaga/.cache/huggingface/datasets/csv/default-f7f202862046c4cb/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [6]:
dataset_generated.features

{'Cover Letter': Value(dtype='string', id=None),
 'Job Description': Value(dtype='string', id=None),
 'Linked Resume': Value(dtype='string', id=None)}

In [7]:
dataset_generated = load_dataset('csv', data_files=path_file, split='train[:80%]')

Found cached dataset csv (C:/Users/Jayathilaga/.cache/huggingface/datasets/csv/default-f7f202862046c4cb/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [8]:
dataset_generated

Dataset({
    features: ['Cover Letter', 'Job Description', 'Linked Resume'],
    num_rows: 273
})

In [9]:
dev_set = load_dataset('csv', data_files=path_file, split='train[81%:]')

Found cached dataset csv (C:/Users/Jayathilaga/.cache/huggingface/datasets/csv/default-f7f202862046c4cb/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [10]:
dataset_generated

Dataset({
    features: ['Cover Letter', 'Job Description', 'Linked Resume'],
    num_rows: 273
})

In [11]:
def add_prefix(data):
    data['Job description and Linked Resume'] = "give me a cover letter based on the a job desciption and a resume. Job desciption:"+data['Job Description'] +"Resume:"+ data['Linked Resume']
    return data 

In [12]:
dataset_generated=dataset_generated.map(add_prefix, remove_columns=["Job Description",'Linked Resume'])

Map:   0%|          | 0/273 [00:00<?, ? examples/s]

In [13]:
dev_set=dev_set.map(add_prefix, remove_columns=["Job Description",'Linked Resume'])

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

In [14]:
# tokenized_inputs[0]

In [15]:
from datasets import concatenate_datasets


tokenized_inputs = dataset_generated.map(lambda x: tokenizer(x["Job description and Linked Resume"], truncation=True), batched=True
                                         , remove_columns=['Cover Letter', 'Job description and Linked Resume'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = dataset_generated.map(lambda x: tokenizer(x["Cover Letter"], truncation=True), batched=True
                                          , remove_columns=['Cover Letter', 'Job description and Linked Resume'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/273 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/273 [00:00<?, ? examples/s]

Max target length: 512


In [16]:
#for the dev set

tokenized_inputs_dev = dev_set.map(lambda x: tokenizer(x["Job description and Linked Resume"], truncation=True), batched=True
                                         , remove_columns=['Cover Letter', 'Job description and Linked Resume'])
max_source_length_dev = max([len(x) for x in tokenized_inputs_dev["input_ids"]])
print(f"Max source length: {max_source_length_dev}")

# The maximum total sequence length for target text after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets_dev = dev_set.map(lambda x: tokenizer(x["Cover Letter"], truncation=True), batched=True
                                          , remove_columns=['Cover Letter', 'Job description and Linked Resume'])
max_target_length_dev = max([len(x) for x in tokenized_targets_dev["input_ids"]])
print(f"Max target length: {max_target_length_dev}")

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/65 [00:00<?, ? examples/s]

Max target length: 480


In [17]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = [ item for item in sample["Job description and Linked Resume"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["Cover Letter"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_dataset = dataset_generated.map(preprocess_function, batched=True, remove_columns=['Cover Letter', 'Job description and Linked Resume'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset.features)}")

Map:   0%|          | 0/273 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [18]:
dataset_generated

Dataset({
    features: ['Cover Letter', 'Job description and Linked Resume'],
    num_rows: 273
})

In [19]:
def preprocess_function_dev(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = [ item for item in sample["Job description and Linked Resume"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length_dev, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["Cover Letter"], max_length=max_target_length_dev, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_dataset_dev = dev_set.map(preprocess_function, batched=True, remove_columns=['Cover Letter', 'Job description and Linked Resume'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset_dev.features)}")

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [20]:
dev_set

Dataset({
    features: ['Cover Letter', 'Job description and Linked Resume'],
    num_rows: 65
})

In [22]:
!pip install rouge-score

  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=36f2f75be62066697d0d4957452cc0de02827c3e81e0386853ad34b576ccd3c5
  Stored in directory: c:\users\jayathilaga\appdata\local\pip\cache\wheels\9b\3d\39\09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [23]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jayathilaga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [25]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [33]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
model_id="google/flan-t5-large"
# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-covergenie"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,gradient_accumulation_steps=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    # num_train_epochs=5,
    optim='adafactor',
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=50,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=50,
    load_best_model_at_end=True,
    gradient_checkpointing=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    max_steps=100,
    eval_steps =50,
    metric_for_best_model ='eval_loss',
    greater_is_better =False,
    # hub_model_id=repository_id,
    # hub_token=HfFolder.get_token(),
)


In [38]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset_dev,
    compute_metrics=compute_metrics,
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 6.00 GiB total capacity; 5.30 GiB already allocated; 0 bytes free; 5.32 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.train()

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pwd

/content


In [ ]:
trainer.save_model("my_model") 

In [ ]:
loaded_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/my_model")

In [ ]:
trainer

In [ ]:
loaded_model

In [31]:
JD='The medical records clerk will be responsible for organizing and maintaining the medical records of the patients at the hospital. The clerk will work with the nursing staff to ensure that the records are accurate and up to date. The clerk will also work with the doctors and other medical staff to ensure that the records are complete and accurate. The clerk will also be responsible for keeping the records confidential and keeping them safe from unauthorized access.'

In [32]:
resume_text= 'Amy Chan Medical Records Clerk -3 years of experience working in medical records departments in hospitals -Able to work well with other staff members and doctors to ensure accuracy of records -Confidential and able to keep records safe from unauthorized access -Nursing degree from St. Helena University'

In [ ]:
final_text="give me a cover letter based on the a job desciption and a resume. Job desciption:"+JD +" Resume:"+ resume_text

In [ ]:
max_source_length=512


In [ ]:
final_text

'give me a cover letter based on the a job desciption and a resume. Job desciption:The medical records clerk will be responsible for organizing and maintaining the medical records of the patients at the hospital. The clerk will work with the nursing staff to ensure that the records are accurate and up to date. The clerk will also work with the doctors and other medical staff to ensure that the records are complete and accurate. The clerk will also be responsible for keeping the records confidential and keeping them safe from unauthorized access. Resume:Amy Chan Medical Records Clerk -3 years of experience working in medical records departments in hospitals -Able to work well with other staff members and doctors to ensure accuracy of records -Confidential and able to keep records safe from unauthorized access -Nursing degree from St. Helena University'

In [ ]:
inputs = tokenizer(final_text, max_length=max_source_length, truncation=True, return_tensors="pt")
output = loaded_model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=max_target_length)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)

Amy Chan (111) 789-3456 amy.chan@email.com 20-Aug-19 Dear Hiring Manager, I'm excited to apply for the Medical Records Clerk position at St. Helena Hospital.


In [ ]:
from transformers import GenerationConfig

In [ ]:
generation_config = GenerationConfig.from_pretrained("google/flan-t5-large",temperature=2.0)

In [ ]:
# generation_config = GenerationConfig.from_pretrained("./my_model")

In [ ]:
inputs = tokenizer(final_text, max_length=max_source_length, truncation=True, return_tensors="pt")
output = loaded_model.generate(**inputs, num_beams=3, do_sample=True, min_length=500,
                               max_length=max_target_length,generation_config=generation_config,num_return_sequences=3)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

KeyboardInterrupt: ignored

In [ ]:
print(predicted_title)

In [ ]:
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]
print(predicted_title)

In [ ]:
 nltk.sent_tokenize(tokenizer.batch_decode(output, skip_special_tokens=True)[0])